## My train using Learner

Training BART with a changed script from the BERT project (adding position_ids)

### !! PROBLEMS
Miska call
* chyba, ze mam jen jeden tokenizer.... PROC?
* generate nezere position_ids 
* zkusit tomu nedavat decoder_input_ids, jenom labels (melo by si je to snad dopocitat z labels)... JO TO ASI POMOHLO
    * prepare_decoder_input_ids_from_labels
    * pokud nejsou shiftnuty labely doleva, vysvetluje to nizkou loss

In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)


import argparse
from datetime import datetime
import json
import os
import pickle
import random
import time
import wandb
from pathlib import Path
import gc
import glob
from tqdm import tqdm

import numpy as np
from transformers import TrainingArguments, Trainer, BartConfig, BartForConditionalGeneration
from transformers.file_utils import logging
# from tensorboardX import SummaryWriter
import torch

# custom veci
from dataset import SpectroDataset, SpectroDataCollator
sys.path.append('data')
sys.path.append('bart_spektro')
from modeling_bart_spektro import BartSpektoForConditionalGeneration
from configuration_bart_spektro import BartSpektroConfig
from data_preprocess1 import print_args
from bart_spektro_tokenizer import BartSpektroTokenizer
from tokenizers import Tokenizer

# from utils import add_special_tokens, generate_sample, sample_seq, set_seed, top_k_top_p_filtering, print_args

#### Setting basic training args

In [2]:
!pwd

/mnt/storage-brno6/home/ahajek/Spektro/MassGenie


In [3]:
bs = 32 #4
gas = 16/int(os.environ["PBS_NGPUS"]) #16
print("GA:", gas)
which_bart = "spektro" #"original" # "spektro"
data_type = "8M"
tokenizer_type = "_bbpe_1M" # for spektro tokenizer use ""
tokenizer = Tokenizer.from_file(f"./tokenizer/bbpe_tokenizer/bart{tokenizer_type}_tokenizer.model")

SEQ_LEN = 200
num_epochs = 100 # 10 (BARTy se trenovaly 10 epoch celkem) # int(os.environ["TOTAL_EPOCHS"])
resume_training = False # bool(int(os.environ["RESUME_TRAINING"]))
resume_wandb_id = "" # "1l7305qk" #pass # ""

model = None # aby nebyl nedefinovany

# find the last checkpoint

# models_pth = "/storage/projects/msml/mg_neims_branch/models/bart_trial"
# runs = glob.glob(models_pth)
# checkpoints =  glob.glob(runs[-1]+"/checkpoint-*")
# checkpoints.sort(key=lambda x: int(x.split("-")[-1]))
# load_checkpoint = checkpoints[-1]
# print(f"last checkpoint: {load_checkpoint}")

# load_checkpoint = "./models/bart_2022-06-01-04_30_20/checkpoint-4152/"

#### Setting all training args

In [3]:
os.environ["PBS_NGPUS"]

KeyError: 'PBS_NGPUS'

In [5]:
now = str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
now = now.replace(":","_").replace(" ", "-")
parser = argparse.ArgumentParser()
parser.add_argument("--lr",default=5e-5, type=float, help="learning rate")
parser.add_argument("--seed",default=42, type=int,  help="seed to replicate results")
parser.add_argument("--gradient-accumulation-steps",default=gas, type=int, help="gradient_accumulation_steps")
parser.add_argument("--batch-size",default=bs, type=int,  help="batch_size")
parser.add_argument("--warmup",default=500, type=int,  help="warmup steps for learning rate")
parser.add_argument("--weight-decay",default=0.01, type=float,  help="weight decay rate parameter")
parser.add_argument("--n-gpu",default=os.environ["PBS_NGPUS"], type=int, required=False, help="no of gpu available")
parser.add_argument("--fairscale",default="", type=str, required=False, choices=["simple", "zero_dp_2", "zero_dp_3"], help="GPU paralellization via Fairscale, " +
                    "more info in HuggingFace's Trainer docs")
parser.add_argument("--deepspeed", default=None, type=str, required=False, help="GPU paralellization via Deepspeed, the value is the location of DeepSpeed json config file; " +
                    "more info in HuggingFace's Trainer docs")
parser.add_argument("--num-workers",default=os.environ["PBS_NCPUS"], type=int,  help="num of cpus available")
parser.add_argument("--device",default=torch.device('cuda'), help="torch.device object")
parser.add_argument("--num-train-epochs",default=num_epochs, type=int,  help="number of training epochs")
parser.add_argument("--output-dir",default='/storage/projects/msml/mg_neims_branch/MassGenie/output', type=str,  help="Path to save evaluation results")
parser.add_argument("--save-dir",default='/storage/projects/msml/mg_neims_branch/MassGenie/models', type=str,  help="Path to save trained model")
parser.add_argument("--save-name", type=str, default=f'bart_{now}', help="Name of the model, used for saves")
parser.add_argument("--load-checkpoint", type=str, default='', help="Path to the checkpoint to resume training")
parser.add_argument("--config-dir",default='/storage/projects/msml/mg_neims_branch/MassGenie/configs', type=str,  help="Path to save config files of models")
parser.add_argument("--fp16",default=True, type=bool, required=False, help="whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit")
parser.add_argument("--max-grad-norm",default=1.0, type=float, help="max gradient norm.")
parser.add_argument("--train-data-path",default=f'/storage/projects/msml/mg_neims_branch/MassGenie/data/trial_set/{data_type}{tokenizer_type}_bart_prepared_data_train.pkl', type=str, help="Path to jsonl train dataset")
parser.add_argument("--valid-data-path",default=f'/storage/projects/msml/mg_neims_branch/MassGenie/data/trial_set/{data_type}{tokenizer_type}_bart_prepared_data_valid.pkl', type=str, help="Path to jsonl validation dataset")
parser.add_argument("--log-steps",default=50, type=int,  help="number of steps between logs")
parser.add_argument("--eval-steps",default=7142, type=int,  help="number of steps between evaluations")
parser.add_argument("--tokenizer-path",default='/storage/brno6/home/ahajek/nic', type=str, help="location of the desied tokenizer (special sep token will be added))")
parser.add_argument("--model-path",default='/storage/projects/msml/mg_neims_branch/MassGenie/models/NECO', type=str, help="location of the desired model to finetune")
parser.add_argument("--wandb", action='store_true', default=True, help="optinal logging via Weights&Biases")
parser.add_argument("--wandb-resume", action='store_true', default=resume_training, help="resume logging via wandb, needs an valid run ID set in args.wandb-id")
parser.add_argument("--wandb-id", type=str, default=wandb.util.generate_id(), help="Process unique wandb ID used for resumin the training process")
parser.add_argument("--tensorboard", action='store_true', default=False, help="optinal logging via TensorBoard")

args = parser.parse_args([])
arg_log = print_args(args)

# extended outputs
logging.set_verbosity_info()

arguments:
  lr ........................... 5e-05
  seed ......................... 42
  gradient_accumulation_steps .. 16
  batch_size ................... 32
  warmup ....................... 500
  weight_decay ................. 0.01
  n_gpu ........................ 1
  fairscale .................... 
  deepspeed .................... None
  num_workers .................. 5
  device ....................... cuda
  num_train_epochs ............. 10
  output_dir ................... /storage/projects/msml/mg_neims_branch/MassGenie/output
  save_dir ..................... /storage/projects/msml/mg_neims_branch/MassGenie/models
  save_name .................... bart_2022-06-27-18_17_59
  load_checkpoint .............. 
  config_dir ................... /storage/projects/msml/mg_neims_branch/MassGenie/configs
  fp16 ......................... True
  max_grad_norm ................ 1.0
  train_data_path .............. /storage/projects/msml/mg_neims_branch/MassGenie/data/trial_set/8M_bbpe_1M_bart_pre

#### Loading data, tokenizer, model

In [6]:
# BART CONIGURATION
if which_bart == "spektro":
    config = BartSpektroConfig(vocab_size = len(tokenizer.get_vocab()),
                                 max_position_embeddings = SEQ_LEN,
                                 max_length = SEQ_LEN,
                                 min_len = 0,
                                 encoder_layers = 12,
                                 encoder_ffn_dim = 4096,
                                 encoder_attention_heads = 16,
                                 decoder_layers = 12,
                                 decoder_ffn_dim = 4096,
                                 decoder_attention_heads = 16,
                                 encoder_layerdrop = 0.0,
                                 decoder_layerdrop = 0.0,
                                 activation_function = 'gelu',
                                 d_model = 1024,
                                 dropout = 0.2,
                                 attention_dropout = 0.0,
                                 activation_dropout = 0.0,
                                 init_std = 0.02,
                                 classifier_dropout = 0.0,
                                 scale_embedding = False,
                                 use_cache = True,
                                 pad_token_id = 2,
                                 bos_token_id = 3,
                                 eos_token_id = 0,
                                 is_encoder_decoder = True,
                                 decoder_start_token_id = 3,
                                 forced_eos_token_id = 0,
                                 max_log_id=9)

if which_bart == "original":
    config = BartConfig(vocab_size = len(tokenizer.get_vocab()),
                                 max_position_embeddings = SEQ_LEN,
                                 max_length = SEQ_LEN,
                                 min_len = 0,
                                 encoder_layers = 12,
                                 encoder_ffn_dim = 4096,
                                 encoder_attention_heads = 16,
                                 decoder_layers = 12,
                                 decoder_ffn_dim = 4096,
                                 decoder_attention_heads = 16,
                                 encoder_layerdrop = 0.0,
                                 decoder_layerdrop = 0.0,
                                 activation_function = 'gelu',
                                 d_model = 1024,
                                 dropout = 0.2,
                                 attention_dropout = 0.0,
                                 activation_dropout = 0.0,
                                 init_std = 0.02,
                                 classifier_dropout = 0.0,
                                 scale_embedding = False,
                                 use_cache = True,
                                 pad_token_id = 2,
                                 bos_token_id = 3,
                                 eos_token_id = 0,
                                 is_encoder_decoder = True,
                                 decoder_start_token_id = 3,
                                 forced_eos_token_id = 0)

In [7]:
# DATA
train_data = SpectroDataset(args.train_data_path, original=which_bart=="original")
valid_data = SpectroDataset(args.valid_data_path, original=which_bart=="original")

# clean memory
gc.collect()
torch.cuda.empty_cache()

# TOKENIZER
# tokenizer = add_special_tokens(args.tokenizer_path)

# MODEL
if which_bart == "original":
    model = BartForConditionalGeneration(config)
else:
    model = BartSpektoForConditionalGeneration(config)
# model = BartForConditionalGeneration.from_pretrained(args.model_path)
model.to(args.device)

print("kratkej vypis :D")

kratkej vypis :D


In [8]:
# for 1M dataset
# train_data.data.rename(columns={"lm_labels":"labels"}, inplace=True)
# valid_data.data.rename(columns={"lm_labels":"labels"}, inplace=True)
if which_bart == "original":
    try:
        train_data.data.drop(columns=["position_ids"], inplace=True)
        valid_data.data.drop(columns=["position_ids"], inplace=True)
        train_data.data.drop(columns=["decoder_input_ids"], inplace=True)
        valid_data.data.drop(columns=["decoder_input_ids"], inplace=True)
    except:
        pass

In [9]:
assert 0 == sum([len(x) != SEQ_LEN for x in train_data.data.input_ids])
# assert 0 == sum([len(x) != SEQ_LEN for x in train_data.data.decoder_input_ids])
assert 0 == sum([len(x) != SEQ_LEN for x in train_data.data.encoder_attention_mask])
assert 0 == sum([len(x) != SEQ_LEN for x in train_data.data.decoder_attention_mask])
assert 0 == sum([len(x) != SEQ_LEN for x in train_data.data.labels])
if which_bart == "spektro":
    assert 0 == sum([len(x) != SEQ_LEN for x in train_data.data.position_ids])
    
# NA TOTO BACHAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA  POZOOOOOOOOOOOOOOOOOR POZOROROROROROROROROROORORORORORKVOKVOEK    
# train_data.data.drop(columns=["decoder_input_ids"], inplace=True)
# valid_data.data.drop(columns=["decoder_input_ids"], inplace=True)
train_data.data

,destereo_smiles,input_ids,decoder_input_ids,encoder_attention_mask,decoder_attention_mask,labels,position_ids
0,c1ccc(cc1)CNN=Nc1ccccc1,"[14, 15, 26, 27, 28, 29, 30, 32, 33, 37, 38, 3...","[3, 224, 70, 20, 280, 11, 266, 20, 12, 679, 32...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 224, 70, 20, 280, 11, 266, 20, 12, 679, 32...","[2, 0, 3, 5, 6, 3, 5, 1, 2, 3, 5, 8, 6, 7, 6, ..."
1,O=C1CCC2C(C3C1C3(C)C)C(CC2)(C)O,"[26, 27, 28, 29, 30, 31, 37, 38, 39, 40, 41, 4...","[3, 224, 50, 32, 38, 20, 276, 21, 38, 11, 38, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 224, 50, 32, 38, 20, 276, 21, 38, 11, 38, ...","[5, 8, 6, 8, 5, 6, 1, 5, 8, 7, 9, 7, 9, 6, 6, ..."
2,NC(=S)C1(CCC1)COc1ccccc1,"[14, 15, 16, 17, 18, 25, 26, 27, 28, 29, 30, 3...","[3, 224, 265, 260, 54, 12, 38, 20, 11, 276, 20...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 224, 265, 260, 54, 12, 38, 20, 11, 276, 20...","[1, 3, 2, 3, 3, 1, 5, 7, 6, 6, 4, 4, 2, 4, 4, ..."
3,COc1cccc2c1n(CC(=O)C)c(cc2=O)C,"[14, 15, 26, 27, 29, 31, 33, 37, 38, 39, 40, 4...","[3, 224, 327, 20, 338, 21, 70, 20, 81, 11, 261...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 224, 327, 20, 338, 21, 70, 20, 81, 11, 261...","[0, 2, 1, 1, 3, 1, 3, 4, 6, 7, 6, 7, 7, 9, 7, ..."
4,Clc1cccc(c1)n1c(=O)cc([nH]c1=O)C(=O)O,"[25, 26, 29, 33, 34, 35, 36, 37, 38, 39, 40, 4...","[3, 224, 364, 70, 20, 338, 11, 70, 20, 12, 81,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 224, 364, 70, 20, 338, 11, 70, 20, 12, 81,...","[1, 2, 0, 1, 3, 6, 7, 7, 8, 8, 8, 8, 8, 7, 9, ..."
...,...,...,...,...,...,...,...
4641073,Oc1ccc(cc1)C1CCN(CC1)C(=O)c1ccco1,"[33, 34, 37, 38, 39, 40, 41, 42, 43, 44, 45, 4...","[3, 224, 469, 20, 280, 11, 266, 20, 12, 38, 20...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 224, 469, 20, 280, 11, 266, 20, 12, 38, 20...","[3, 3, 4, 6, 8, 6, 7, 7, 6, 6, 4, 3, 5, 7, 6, ..."
4641074,CC(=O)Nc1ccc(cc1)C(c1ccc(cc1)C)(C)C,"[33, 34, 38, 39, 40, 41, 42, 43, 44, 45, 50, 5...","[3, 224, 261, 260, 50, 12, 329, 20, 280, 11, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 224, 261, 260, 50, 12, 329, 20, 280, 11, 2...","[2, 2, 3, 7, 6, 8, 5, 9, 5, 2, 6, 7, 6, 7, 3, ..."
4641075,Brc1ccc(cc1)C(c1ccncc1)N,"[14, 15, 17, 18, 25, 26, 27, 28, 29, 30, 34, 3...","[3, 224, 466, 70, 20, 280, 11, 266, 20, 12, 38...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 224, 466, 70, 20, 280, 11, 266, 20, 12, 38...","[0, 2, 2, 1, 1, 5, 5, 6, 4, 3, 1, 3, 6, 7, 7, ..."
4641076,OC(=O)c1cc(nc2c1c(C)no2)c1ccccc1F,"[33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 4...","[3, 224, 286, 260, 50, 12, 70, 20, 266, 11, 27...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 224, 286, 260, 50, 12, 70, 20, 266, 11, 27...","[6, 3, 5, 6, 7, 9, 8, 8, 8, 8, 9, 9, 6, 3, 0, ..."


In [10]:
# repair type at position_ids

#for x in train_data.data.position_ids:
# #     print(x.dtype)
#     if x.dtype != np.dtype('int32'):
#         print(x)
# train_data.data.position_ids[0].dtype == np.dtype('float64')

In [11]:
print(f"Number of model parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

Number of model parameters: 354200576


#### Setting run resuming and WandB logging

In [12]:
# Resume training
if resume_training:
    args.load_checkpoint = load_checkpoint
    if args.wandb_resume:
        args.wandb_id = resume_wandb_id

# Init wandb
if args.wandb:
    wandb.login()
    wandb.init(id=args.wandb_id, resume="allow", entity="hajekad", project="BART_for_gcms")
    wandb.run.name = args.save_name

wandb: Currently logged in as: msgc_boys (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.19 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


#### Setting training arguments (according to args)

In [13]:
training_args = TrainingArguments(
    output_dir=args.save_dir+"/"+args.save_name,         # output directory
    num_train_epochs=args.num_train_epochs,              # total # of training epochs
    per_device_train_batch_size=args.batch_size,         # batch size per device during training
    per_device_eval_batch_size=args.batch_size,          # batch size for evaluation
    gradient_accumulation_steps=args.gradient_accumulation_steps,
    warmup_steps=args.warmup,                                    # number of warmup steps for learning rate scheduler
    weight_decay=args.weight_decay,                                   # strength of weight decay
    logging_dir=args.save_dir + './logs',                # directory for storing logs
    report_to=["wandb"],
    run_name=args.save_name,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
#     eval_steps=args.eval_steps,
    logging_strategy="steps",
    logging_steps=1, # args.log_steps,
    save_strategy="epoch",
#     save_steps=1000,
    fp16=args.fp16,
    dataloader_drop_last=True,
    save_total_limit=2,
    dataloader_num_workers=args.num_workers,
    sharded_ddp=args.fairscale,
    deepspeed=args.deepspeed,
    load_best_model_at_end = True
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_data,            # training dataset
    eval_dataset=valid_data,             # evaluation dataset
    data_collator = SpectroDataCollator(original=(which_bart=="original"))
)

PyTorch: setting up devices
Using amp half precision backend


### Training

In [ ]:
# log args to "args" file in args.save_dir (right before the training starts, to be the latest)
Path(f'{args.save_dir}/{args.save_name}').mkdir(exist_ok=True)
with open(f'{args.save_dir}/{args.save_name}/args', 'w+') as output_file:
    output_file.write(arg_log)

if args.load_checkpoint:
    trainer.train(args.load_checkpoint)
else:
    #trainer.evaluate()
    trainer.train()

/storage/brno6/home/ahajek/.local-Pytorch-21.SIF/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4641078
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 16
  Total optimization steps = 90640
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:752: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please us

Epoch,Training Loss,Validation Loss


### Inference

In [9]:
from tokenizers import Tokenizer

# tokenizer = BartSpektroTokenizer().init_tokenizer()
tok = "./tokenizer/bbpe_tokenizer/bart_bbpe_1M_tokenizer.model"
tokenizer = Tokenizer.from_file(tok)

#### Generate and display SMILES from valid data and show the loss

In [30]:
from transformers import AutoModel
import glob 

# load model if not defined
model=None     # force the model loading
if not model:
    models_pth = "/storage/projects/msml/mg_neims_branch/MassGenie/models/*"
    runs = glob.glob(models_pth)
    print(runs)
    checkpoints =  glob.glob(sorted(runs)[-3]+"/checkpoint-*")
    checkpoints.sort(key=lambda x: int(x.split("-")[-1]))
    print(checkpoints)
    load_checkpoint = checkpoints[-1]
    print(f"last checkpoint: {load_checkpoint}")
    model = BartSpektoForConditionalGeneration.from_pretrained(load_checkpoint)
    model.to(args.device)

loading configuration file /storage/projects/msml/mg_neims_branch/MassGenie/models/bart_2022-05-31-11_53_31_bbpe1K_IDs/checkpoint-6920/config.json
Model config BartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "BartSpektoForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 503,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 502,
  "dropout": 0.2,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 502,
  "forced_eos_token_id": 502,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_log_id": 9,
  "max_position_embeddings": 200,
  "model_type"

['/storage/projects/msml/mg_neims_branch/MassGenie/models/bart_2022-05-11-14_44_04_NO_P_IDs', '/storage/projects/msml/mg_neims_branch/MassGenie/models/bart_2022-06-01-09_52_21_bbpe1M_IDs', '/storage/projects/msml/mg_neims_branch/MassGenie/models/bart_2022-06-01-14_50_36', '/storage/projects/msml/mg_neims_branch/MassGenie/models/bart_2022-05-31-11_53_31_bbpe1K_IDs', '/storage/projects/msml/mg_neims_branch/MassGenie/models/bart_2022-05-10-16_06_04_P_IDs']
['/storage/projects/msml/mg_neims_branch/MassGenie/models/bart_2022-05-31-11_53_31_bbpe1K_IDs/checkpoint-6228', '/storage/projects/msml/mg_neims_branch/MassGenie/models/bart_2022-05-31-11_53_31_bbpe1K_IDs/checkpoint-6920']
last checkpoint: /storage/projects/msml/mg_neims_branch/MassGenie/models/bart_2022-05-31-11_53_31_bbpe1K_IDs/checkpoint-6920


All model checkpoint weights were used when initializing BartSpektoForConditionalGeneration.

All the weights of BartSpektoForConditionalGeneration were initialized from the model checkpoint at /storage/projects/msml/mg_neims_branch/MassGenie/models/bart_2022-05-31-11_53_31_bbpe1K_IDs/checkpoint-6920.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BartSpektoForConditionalGeneration for predictions without further training.


In [31]:
generated[0]

[502,
 503,
 224,
 437,
 260,
 50,
 12,
 263,
 20,
 269,
 11,
 261,
 20,
 12,
 38,
 260,
 50,
 12,
 38,
 502]

In [35]:
torch.seed()
with torch.no_grad():
    model.eval()
    inputs = valid_data[500] # 489
    outputs = model(input_ids = inputs["input_ids"].unsqueeze(0).to(device=args.device), 
                    position_ids = inputs["position_ids"].unsqueeze(0).to(device=args.device),
                    labels = inputs["labels"].unsqueeze(0).to(device=args.device))
    generated = model.generate(
               input_ids=inputs["input_ids"].unsqueeze(0).to(device=args.device), 
               position_ids=inputs["position_ids"].unsqueeze(0).to(device=args.device),
               bos_token_id=503,
               forced_bos_token_id=503,
               top_p=0.5,
               top_k=50,
               min_length=30,
               do_sample=True,
               temperature=0.7
               ).tolist()[0],
               


# print("ground truth:", tokenizer.ids_to_smiles(inputs["labels"].tolist()), "\n\n")
# print("ground truth:", tokenizer.decode(inputs["labels"].tolist()), "\n\n")
print("ground truth ids:", tokenizer.decode(np.array(inputs["labels"].tolist())*np.array(inputs["decoder_attention_mask"].tolist())), "\n\n")
print("mask:", inputs["decoder_attention_mask"].tolist(), "\n\n")
print("generated:", generated, "\n")
# print("generated:", tokenizer.ids_to_smiles(generated[0]), "\n")
print("generated:", tokenizer.decode(generated[0]), "\n")
print(f"loss: {outputs.loss}")
print(f"logit bos: {outputs.logits[0][68][503].cpu().numpy()}")

ground truth ids:  COc1cccc(c1)CC(=O)NC1CN(CC1O)C(=O)C(NC(=O)N)C 


mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 


generated: ([502, 503, 224, 331, 20, 323, 11, 70, 20, 12, 261, 260, 50, 12, 272, 20, 266, 11, 261, 502],) 

generated: COCCCSNCCOCCO OCCO1ccnc(c1)CC(=O)CCO1cc(CCCOCCCS 

loss: 4.943031311035156
logit bos: 0.5182749629020691


In [ ]:
output = model.forward(input_ids = inputs["input_ids"].unsqueeze(0).to(device=args.device), 
                    position_ids = inputs["position_ids"].unsqueeze(0).to(device=args.device),
                    labels = inputs["labels"].unsqueeze(0).to(device=args.device))

In [39]:
output.loss

tensor(3.0152, device='cuda:0', grad_fn=<NllLossBackward>)

In [40]:
k = 10
x = outputs.logits[0][10].cpu().numpy()
top_k = np.partition(x, -k)[-k:]
outputs

Seq2SeqLMOutput(loss=tensor(3.0152, device='cuda:0'), logits=tensor([[[-0.0170, -0.3663, -0.1512,  ..., -0.6392, -1.5668, -0.9503],
         [ 0.5132, -1.0161,  0.2738,  ...,  0.1879, -1.1371, -0.7397],
         [ 0.4215, -2.6604, -2.6799,  ..., -2.3999, -2.8463, -2.9328],
         ...,
         [12.8004, -0.0289,  0.2292,  ...,  0.1463, -0.3587, -0.5953],
         [12.8562,  0.0174,  0.2741,  ...,  0.1814, -0.3457, -0.5184],
         [13.1502,  0.0494,  0.3001,  ...,  0.1577, -0.2962, -0.5157]]],
       device='cuda:0'), past_key_values=None, decoder_hidden_states=None, decoder_attentions=None, cross_attentions=None, encoder_last_hidden_state=tensor([[[-0.7694, -0.9320, -1.0044,  ...,  0.3718,  0.7595,  0.8037],
         [-0.7321, -0.9464, -0.9625,  ...,  0.3598,  0.7415,  0.7888],
         [ 0.8197,  0.9224, -0.6806,  ...,  0.7534, -0.3882,  1.7202],
         ...,
         [ 0.0690,  0.5991, -1.3737,  ...,  0.0799, -0.0585,  1.4387],
         [ 0.0690,  0.5991, -1.3737,  ...,  0.0799

### Run evalutaion

,destereo_smiles,input_ids,decoder_input_ids,encoder_attention_mask,decoder_attention_mask,lm_labels,position_ids
9,OCC1OC(Oc2ccccc2C(=O)OC)C(C(C1O)O)NC(=O)C,"[14, 15, 16, 17, 18, 19, 26, 27, 28, 29, 30, 3...","[503, 542, 531, 531, 520, 542, 531, 506, 542, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 542, 531, 531, 520, 542, 531, 506, 542, ...","[2, 6, 1, 4, 5, 5, 3, 6, 6, 6, 5, 8, 5, 3, 6, ..."
15,OC(=O)C(CSCC1OC(C(C1O)O)n1cnc2c1ncnc2N)N,"[16, 17, 18, 19, 25, 26, 27, 28, 29, 30, 31, 3...","[503, 542, 531, 506, 510, 542, 507, 531, 506, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 542, 531, 506, 510, 542, 507, 531, 506, ...","[1, 1, 3, 0, 1, 1, 4, 5, 5, 4, 0, 1, 2, 2, 6, ..."
16,OCC1OC(SC2OC(CO)C(C(C2O)O)O)C(C(C1O)O)O,"[14, 15, 16, 17, 18, 19, 25, 26, 27, 28, 29, 3...","[503, 542, 531, 531, 520, 542, 531, 506, 545, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 542, 531, 531, 520, 542, 531, 506, 545, ...","[3, 4, 3, 4, 6, 4, 4, 6, 7, 6, 8, 6, 8, 5, 4, ..."
31,O=C1NC(C(=O)N1)CC(=O)N1CCN(CC1)CC(=O)N1CCCCC1,"[28, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 4...","[503, 542, 510, 531, 520, 541, 531, 506, 531, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 542, 510, 531, 520, 541, 531, 506, 531, ...","[3, 4, 3, 6, 4, 6, 7, 7, 8, 9, 8, 9, 6, 1, 2, ..."
41,COC(=O)C(CC(=O)OC)NCCCNC(C(=O)OC)CC(=O)OC,"[15, 25, 26, 27, 28, 29, 30, 31, 32, 33, 38, 3...","[503, 531, 542, 531, 506, 510, 542, 507, 531, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 531, 542, 531, 506, 510, 542, 507, 531, ...","[6, 1, 5, 6, 7, 6, 4, 6, 3, 6, 1, 6, 6, 8, 9, ..."
75,O=c1cc(N2CCC(C2)NS(=O)(=O)N2CCCC2)n(c(=O)n1C)C,"[27, 28, 33, 34, 36, 38, 39, 40, 41, 42, 43, 4...","[503, 542, 510, 555, 520, 555, 555, 506, 541, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 542, 510, 555, 520, 555, 555, 506, 541, ...","[0, 2, 4, 3, 1, 3, 7, 6, 8, 9, 8, 7, 6, 5, 3, ..."
76,COCC(=O)N1CCN(C2C1CS(=O)(=O)C2)C(=O)c1cnn(c1)C,"[33, 34, 36, 38, 39, 40, 41, 42, 43, 44, 45, 4...","[503, 531, 542, 531, 531, 506, 510, 542, 507, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 531, 542, 531, 531, 506, 510, 542, 507, ...","[4, 1, 1, 5, 6, 7, 8, 9, 8, 8, 8, 4, 1, 6, 5, ..."
99,OCC(NC(=O)CC1C(=O)NCCN1Cc1ccc(cc1)C)(CO)CO,"[14, 15, 17, 18, 19, 25, 26, 27, 28, 29, 30, 3...","[503, 542, 531, 531, 506, 541, 531, 506, 510, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 542, 531, 531, 506, 541, 531, 506, 510, ...","[3, 5, 4, 5, 0, 3, 5, 7, 8, 7, 6, 7, 6, 5, 2, ..."
119,O=C(CNC(=O)c1ccccc1)NCC(=O)NC1CCS(=O)(=O)C1,"[16, 17, 18, 28, 29, 30, 32, 33, 36, 37, 38, 3...","[503, 542, 510, 531, 506, 531, 541, 531, 506, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 542, 510, 531, 506, 531, 541, 531, 506, ...","[0, 2, 3, 4, 4, 6, 3, 0, 0, 1, 3, 5, 4, 6, 6, ..."
120,Cn1ncc(c1)N1CC(CCC1=O)C(=O)N1CCNS(=O)(=O)CC1,"[33, 34, 36, 38, 39, 40, 41, 42, 43, 44, 45, 4...","[503, 531, 565, 520, 565, 555, 555, 506, 555, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 531, 565, 520, 565, 555, 555, 506, 555, ...","[3, 1, 0, 4, 7, 6, 8, 9, 8, 8, 7, 1, 1, 3, 6, ..."


In [26]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_data,            # training dataset
    eval_dataset=valid_data,             # evaluation dataset
    data_collator = SpectroDataCollator()
)

Using amp half precision backend


In [22]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 44346
  Batch size = 32


{'eval_loss': 0.14979185163974762,
 'eval_runtime': 123.3335,
 'eval_samples_per_second': 359.562,
 'eval_steps_per_second': 11.238,
 'epoch': 10.0}

In [46]:
# own evaluation 
model.eval()
losses = []
with torch.no_grad():
    for i in tqdm(range(1000)):#tqdm(range(valid_data.len)):
        inputs = valid_data[i]
        outputs = model(input_ids = inputs["input_ids"].unsqueeze(0).to(device=args.device),
                        position_ids = inputs["position_ids"].unsqueeze(0).to(device=args.device),
                        labels = inputs["labels"].unsqueeze(0).to(device=args.device))
        losses.append(outputs.loss.item())
print(f"my evaluation loss: {sum(losses)/len(losses)}")

  6%|▌         | 56/1000 [00:01<00:24, 38.45it/s]


RuntimeError: CUDA error: device-side assert triggered

### Debugging tokenizer

In [20]:
def visualize_tokenizer(t):
    print(f"tok to id: {t.tok_to_id.items()}\n")
    print(f"id to tok: {t.id_to_tok.items()}\n")
    print(f"tl to tok: {t.tl_to_tok.items()}\n")
    print(f"tok to tl: {t.tok_to_tl.items()}\n")

    print(f"unk_tok: {t.unk_tok}")
    print(f"pad_tok: {t.pad_tok}")
    print(f"eos_tok: {t.eos_tok}")
    print(f"bos_tok: {t.bos_tok}")

In [44]:
model.generate(input_ids=inputs["input_ids"].unsqueeze(0).to(device=args.device), 
               position_ids=inputs["position_ids"].unsqueeze(0).to(device=args.device)).tolist()[0]

[502,
 3,
 224,
 300,
 20,
 280,
 11,
 280,
 21,
 280,
 32,
 38,
 11,
 38,
 22,
 38,
 21,
 280,
 22,
 502]

In [22]:
tokenizer.ids_to_smiles([501])

'<pad>'

### Debugging

In [23]:
trainer.state

TrainerState(epoch=29.857142857142858, global_step=90, max_steps=90, num_train_epochs=30, total_flos=5661560773017600.0, log_history=[{'loss': 6.4515, 'learning_rate': 1.0000000000000001e-07, 'epoch': 0.29, 'step': 1}, {'loss': 6.4546, 'learning_rate': 2.0000000000000002e-07, 'epoch': 0.57, 'step': 2}, {'loss': 6.4316, 'learning_rate': 3.0000000000000004e-07, 'epoch': 0.86, 'step': 3}, {'eval_loss': 6.329980373382568, 'eval_runtime': 0.2536, 'eval_samples_per_second': 232.646, 'eval_steps_per_second': 7.886, 'epoch': 0.86, 'step': 3}, {'loss': 9.5658, 'learning_rate': 4.0000000000000003e-07, 'epoch': 1.29, 'step': 4}, {'loss': 6.3142, 'learning_rate': 5.000000000000001e-07, 'epoch': 1.57, 'step': 5}, {'loss': 6.2257, 'learning_rate': 6.000000000000001e-07, 'epoch': 1.86, 'step': 6}, {'eval_loss': 5.930425643920898, 'eval_runtime': 0.2661, 'eval_samples_per_second': 221.705, 'eval_steps_per_second': 7.515, 'epoch': 1.86, 'step': 6}, {'loss': 9.1558, 'learning_rate': 7.000000000000001e-0

In [18]:
!pwd

/mnt/storage-brno6/home/ahajek/Spektro/MassGenie
